## Informações Gerais
 | Informações | Detalhes |
 |------------|-------------|
 |Nome Tabela | silver.psa |
 |Origem | bronze.dados_clientes / bronze.notas_fiscais /bronze.analises_tributarias/ bronze.tarefas_projetos / bronze.logs_sistema |

## Histórico de Atualizações
 | Data | Desenvolvido por | Motivo |
 |:----:|--------------|--------|
 |27/10/2025 | Kálita Boni | Criação do notebook |


In [0]:
# Importações
from pyspark.sql.functions import current_date, current_timestamp, expr

In [0]:
# Funções
#Função que aplica os comentários na tabela
def adicionaComentariosTabela(a,b,c,d):
    spark.sql(f"COMMENT ON TABLE {a}.{b} IS '{c}'")
    for key,value in d.items():
        sqlaux = f"ALTER TABLE {a}.{b} CHANGE COLUMN {key} COMMENT '{value}'"
        spark.sql(sqlaux)

In [0]:
database = "silver"
tabela = "psa_curated.clientes"

In [0]:
df_clientes_silver = spark.sql("""
WITH bronze AS (
    SELECT
        TRIM(id_cliente)                  AS id_cliente,
        TRIM(razao_social)                AS razao_social,
        TRIM(cnpj)                        AS cnpj_raw,
        TRIM(porte_empresa)               AS porte_empresa,
        TRIM(setor)                       AS setor,
        TRIM(cidade)                      AS cidade,
        UPPER(TRIM(estado))               AS estado,
        CAST(data_cadastro AS DATE)       AS data_cadastro
    FROM psa.bronze.dados_clientes
),

curated AS (
    SELECT
        id_cliente,
        -- Padroniza o nome (capitalização e substituição de “Ltda”)
        REGEXP_REPLACE(INITCAP(razao_social), 'Ltda', 'LTDA') AS razao_social,
        
        -- Limpa CNPJ (mantém apenas números)
        LPAD(REGEXP_REPLACE(cnpj_raw, '[^0-9]', ''), 14, '0') AS cnpj,
        
        -- Padroniza os textos
        INITCAP(porte_empresa) AS porte_empresa,
        INITCAP(setor)         AS setor,
        INITCAP(cidade)        AS cidade,
        estado,
        data_cadastro
    FROM bronze
    WHERE id_cliente IS NOT NULL
),

deduplicado AS (
    SELECT * EXCEPT(rn)
    FROM (
        SELECT
            c.*,
            ROW_NUMBER() OVER (
                PARTITION BY id_cliente
                ORDER BY data_cadastro DESC NULLS LAST
            ) AS rn
        FROM curated c
    )
    WHERE rn = 1
)

SELECT
    id_cliente,
    razao_social,
    cnpj,
    porte_empresa,
    setor,
    cidade,
    estado,
    data_cadastro
FROM deduplicado
""")

# Salva como tabela Silver
(df_clientes_silver.write
   .format("delta")
   .mode("overwrite")
   .option("overwriteSchema", "true")
   .saveAsTable("psa_curated.clientes"))

print("✅ Camada Silver criada: psa_curated.clientes")


✅ Camada Silver criada: psa_curated.clientes
